In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [13]:
data_directory = "../Data/Study/B/"
subdirectory = "raw/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_differences(df):
    cleaned_df = clean_df(df)
    difference_df = cleaned_df.loc[1:, ['Movement', 'Time', 'GazeAngleX', 'GazeAngleY']]
    difference_df.reset_index(inplace=True, drop=True)
    difference_df['euclideanDifference'] = difference_df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY'])) - np.array((cleaned_df.loc[row.name, 'GazeAngleX'], cleaned_df.loc[row.name, 'GazeAngleY']))), axis=1)
    difference_df['temporalDifference'] = difference_df.apply(lambda row: (datetime.strptime(row['Time'], "%m/%d/%Y %H:%M:%S.%f") - datetime.strptime(cleaned_df.loc[row.name, 'Time'], "%m/%d/%Y %H:%M:%S.%f")).total_seconds(), axis=1)
    return difference_df

In [10]:
def calculate_precision():
    participantList = os.listdir(data_directory)
    for participant in participantList:
        print(participant)
        if participant ==  '.DS_Store':
            print("skipping calculation")
            continue
        if os.path.exists(data_directory + participant + '/precision/' + subdirectory):
            print("already calculated")
            continue
        fileList = os.listdir(data_directory + participant + '/' + subdirectory)
        for file in fileList:
            if '.csv' in file:
                filePath = data_directory + participant + '/' + subdirectory + file
                df = pd.read_csv(filePath)
                difference_df = calculate_differences(df)
                output_directory = data_directory + participant + '/precision/' + subdirectory
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)
                output_path = output_directory + file
                difference_df.to_csv(output_path, index=False)

In [14]:
calculate_precision()

.DS_Store
skipping calculation
20
11
16
17
21
23
15
12
13
14
22
